# Tutorial 4: Word Embeddings.


### Cuerpo Docente

- Profesores: [Andrés Abeliuk](https://aabeliuk.github.io/), [Felipe Villena](https://fabianvillena.cl/).
- Profesor Auxiliar: María José Zambrano


### Objetivos del Tutorial

- Explicar el problema de la representación Bag of Words.
- Motivación y repaso de qué son los Word Embeddings.
- Explicación de Word2Vec.
- Entrenar nuestros propios `word embeddings` usando un dataset con fuentes de noticias de diversos medios. 💪
- Utilizaremos estos embeddings pre-entrenados para mejorar la capacidad de nuestros modelos en tareas nuevas y en la resuelta en el auxiliar pasado.
- Conocer los `contextualized word embeddings` a través de `BETO`.

## **Motivación**

Partamos por decir que una red neuronal no es más que una serie de operaciones matemáticas sobre vectores con una gran cantidad de dimensiones (tensores). Por ende, si queremos entrenar un modelo necesitamos transformar el texto original a vectores numéricos.

Una de las soluciones más simples a este problema es la representación de Bag of Words (BoW). Si aplicamos este método a cada palabra de cada documento, tendremos un vector one hot encoding por cada palabra. Esto quiere decir que tendremos vectores del largo del vocabulario $V$, con un 1 en la posición asociada a la palabra representada.

Y estamos listos? Podemos entrenar redes neuronales?

La verdad es que no es así, y es que estamos ignorando un gran problema con este enfoque. 😞


### El gran problema de Bag of Words

Pensemos en estas 3 frases como documentos:

- $doc_1$: `¡Buenísima la marraqueta!`
- $doc_2$: `¡Estuvo espectacular ese pan francés!`
- $doc_3$: `!Buenísima esa pintura!`

Sabemos $doc_1$ y $doc_2$ hablan de lo mismo 🍞🍞👌 y que $doc_3$ 🎨 no tiene mucho que ver con los otros.

Supongamos que queremos ver que tan similares son ambos documentos.
Para esto, generamos un modelo `Bag of Words` sobre el documento, aplicando este método por cada palabra para luego tener la representación final.


Es decir, transformamos cada palabra a un vector one-hot y luego los sumamos por documento.

Por simplicidad, omitiremos algunas stopwords y consideramos pan frances como un solo token. Así nos quedaría el siguiente vocabulario:

$$v = \{buenísima, marraqueta, estuvo, espectacular, pan\ francés, pintura\}$$

Entonces, el $\vec{doc_1}$ quedará:

$$\begin{bmatrix}1 \\ 0 \\ 0 \\ 0 \\ 0\\ 0\end{bmatrix} +
  \begin{bmatrix}0 \\ 1 \\ 0 \\ 0 \\ 0\\ 0\end{bmatrix} =
  \begin{bmatrix}1 \\ 1 \\ 0 \\ 0 \\ 0\\ 0\end{bmatrix}$$

El $\vec{doc_2}$ quedará:

$$\begin{bmatrix}0 \\ 0 \\ 1 \\ 0 \\ 0\\ 0\end{bmatrix} +
  \begin{bmatrix}0 \\ 0 \\ 0 \\ 1 \\ 0\\ 0\end{bmatrix} +
  \begin{bmatrix}0 \\ 0 \\ 0 \\ 0 \\ 1\\ 0\end{bmatrix} =
  \begin{bmatrix}0 \\ 0 \\ 1 \\ 1 \\ 1\\ 0\end{bmatrix}$$

Y el $\vec{doc_3}$:

$$\begin{bmatrix}1 \\ 0 \\ 0 \\ 0 \\ 0\\ 0\end{bmatrix} +
  \begin{bmatrix}0 \\ 0 \\ 0 \\ 0 \\ 0\\ 1\end{bmatrix} =
  \begin{bmatrix}1 \\ 0 \\ 0 \\ 0 \\ 0\\ 1\end{bmatrix}$$



**¿Cuál es el problema?**

`buenísima` $\begin{bmatrix}1 \\ 0 \\ 0 \\ 0 \\ 0 \\0\end{bmatrix}$ y `espectacular` $ \begin{bmatrix}0 \\ 0 \\ 0 \\ 1 \\ 0 \\ 0\end{bmatrix}$ representan ideas muy similares. Por otra parte, sabemos que `marraqueta` $\begin{bmatrix}0 \\ 1 \\ 0 \\ 0 \\ 0 \\0\end{bmatrix}$ y `pan francés` $\begin{bmatrix}0 \\ 0 \\ 0 \\ 0 \\ 1 \\0\end{bmatrix}$ se refieren al mismo objeto. Pero en este modelo, estos **son totalmente distintos**. Es decir, los vectores de las palabras que `buenísima` y `espectacular` son tan distintas como `marraqueta` y `pan francés`. Esto se debe a que cada palabra ocupa una dimensión distinta a las demás y son completamente independientes. Esto evidentemente, repercute en la calidad de los modelos que creamos a partir de nuestro Bag of Words.

![BoW](https://raw.githubusercontent.com/dccuchile/CC6205/master/tutorials/recursos/BoW-Problem.png)



Ahora, si queremos ver que documento es mas similar a otro usando distancia euclidiana, veremos que:

$$d(doc_1, doc_2) = 2.236$$
$$d(doc_1, doc_3) = 1.414$$

Es decir, $doc_1$ se parece mas a $doc_3$ aunque nosotros sabemos que $doc_1$ y $doc_2$ nos están diciendo lo mismo!


Nos gustaría que eso no sucediera. Que existiera algún método que nos permitiera hacer que palabras similares tengan representaciones similares. Y que con estas, representemos mejor a los documentos, sin asumir que en el espacio son geométricamente equidistantes, ya que esto no es verdad en la vida real.


--------------------

## **Hipótesis Distribucional**

Estamos buscando algún enfoque que nos permita representar las palabras de forma no aislada, si no como algo que además capture el significado de esta.

Pensemos un poco en la **hipótesis distribucional**. Esta plantea que:

    "Palabras que ocurren en contextos iguales tienden a tener significados similares."

O equivalentemente,

    "Una palabra es caracterizada por la compañía que esta lleva."

Esto nos puede hacer pensar que podríamos usar los contextos de las palabras para generar vectores que describan mejor dichas palabras: en otras palabras, los `Distributional Vectors`.

Por ejemplo, complete la siguiente frase:

Pintaré la muralla de mi casa de color _____

Puede ser rojo, blanco, mostaza, etc..

Son palabras que a uno se les viene a la mente sólo mirando el contexto entregado, por ende podríamos decir que esas son palabras similares, o al menos muy distintas a Murciélago.


### Opción 1: Word-Context Matrix

Es una matriz donde cada celda $(i,j)$ representa la co-ocurrencia entre una palabra objetivo/centro $w_i$ y un contexto $c_j$. El contexto son las palabras dentro de ventana de tamaño $k$ que rodean la palabra central.

Cada fila representa a una palabra a través de su contexto. Como pueden ver, ya no es un vector one-hot, si no que ahora contiene mayor información.

El tamaño de la matriz es el tamaño del vocabulario $V$ al cuadrado. Es decir $|V|*|V|$.

<img src="https://raw.githubusercontent.com/dccuchile/CC6205/master/slides/pics/distributionalSocher.png" alt="Word-context matrices" style="width: 400px;"/>


**Problema: Creada a partir de un corpus respetable, es gigantezca**.

Por ejemplo, para $|v| = 100.000$, la matriz tendrá $\frac{100000 * 100000 * 4}{10^9} = 40gb $. (Recordando que un entero ocupara 4 bytes)

- Es caro mantenerla en memoria
- Los clasificadores no funcionan tan bien con tantas dimensiones (ver [maldición de la dimensionalidad](https://es.wikipedia.org/wiki/Maldici%C3%B3n_de_la_dimensi%C3%B3n)).

**¿Habrá una mejor solución?**

---------------------

### **Word Embeddings**

Es una de las representaciones más populares del vocabulario de un corpus. La idea principal de los Word Embeddings es crear representaciones vectoriales densas y de baja dimensionalidad $(d << |V|)$ de las palabras a partir de su contexto.

Volvamos a nuestro ejemplo anterior: `buenísima` y `espectacular` ocurren muchas veces en el mismo contexto, por lo que los embeddings que los representan debiesen ser muy similares... (*ejemplos de mentira hechos a mano*):

`buenísima` $\begin{bmatrix}0.32 \\ 0.44 \\ 0.92 \\ .001 \end{bmatrix}$ y `espectacular` $\begin{bmatrix}0.30 \\ 0.50 \\ 0.92 \\ .002 \end{bmatrix}$ versus `marraqueta`  $\begin{bmatrix}0.77 \\ 0.99 \\ 0.004 \\ .1 \end{bmatrix}$ el cuál es claramente distinto.


Pero, ¿Cuál es la utilidad de de crear estos vectores en NLP o en el área de Machine Learning en general?

Supongamos que tienen una enfermedad grave y deben ser operados el día de mañana. Le dan a elegir entre ser operados por un estudiante de primer año de medicina con algo de conocimiento médico o bien ser operados por un niño de 5 años 👶. ¿A quién elegirías?

Espero que tu opción haya sido el estudiante con una pequeña noción de los términos médicos implicados en una intervención así. Algo así es lo que se quiso lograr en el [paper](https://arxiv.org/abs/1301.3781) presentado por Mikolov en 2013, aludiendo a la herramienta **Word2Vec**. La idea es que si quieres resolver por ejemplo una tarea de clasificación de texto, ¿no sería útil utilizar el conocimiento de algún modelo pre-entrenado en una tarea similar de texto?. Claro, sería útil partir con los pesos entrenados por otra red, realizando lo que se llama **transfer learning**.

Ya pero.. ¿Cómo generamos estos vectores? ¿Cómo podemos capturar el contexto? ¿Cuál sería esa task auxiliar a utilizar?



##### **Word2vec y Skip-gram**

Word2Vec es probablemente el paquete de software mas famoso para crear word embeddings utilizando distintos modelos que emplean redes neuronales *shallow* o poco profundas.

Este nos provee herramientas para crear distintos tipos de modelos, tales como `Skip-Gram` y `Continuous Bag of Word (CBOW)`. En este caso, solo veremos `Skip-Gram`.

**Skip-gram** es una task auxiliar con la que crearemos nuestros embeddings. Esta tarea involucra tanto a las palabras y al contexto de ellas. Consiste en que por cada palabra del dataset, debemos predecir las palabras de su contexto (las palabras presentes en ventana de algún tamaño $k$).

![Overview](https://raw.githubusercontent.com/dccuchile/CC6205/master/tutorials/recursos/overview-skipgram.png)

Para resolverla, usaremos una red de una sola capa oculta. Los pesos ya entrenados de esta capa serán los que usaremos como embeddings.

#### Detalles del Modelo

- Como dijimos, el modelo será una red de una sola capa. La capa oculta tendrá una dimensión $d$ la cual nosotros determinaremos. Esta capa no tendrá función de activación. Sin embargo, la de salida si, la cual será una softmax para obtener las distribuciones de probabilidades y así ver cuáles palabras pertenecen o no al contexto.

- El vector de entrada, de tamaño $|V|$, será un vector one-hot de la palabra que estemos viendo en ese momento.

- La salida, también de tamaño $|V|$, será un vector que contenga la distribución de probabilidad de que cada palabra del vocabulario pertenezca al contexto de la palabra de entrada.

- Al entrenar, se comparará la distribución de los contextos con la suma de los vectores one-hot del contexto real.


(marraqueta, Estuvo), (marraqueta, buenisima), (marraqueta, la)
![Skip Gram](https://raw.githubusercontent.com/dccuchile/CC6205/master/tutorials/recursos/Skip-gram.png)


Nota: Esto es computacionalmente una locura. Por cada palabra de entrada, debemos calcular la probabilidad de aparición de todas las otras. Imaginen el caso de un vocabulario de 100.000 de palabras y de 10000000 oraciones...

La solución a esto es modificar la task a *Negative Sampling*. Esta transforma este problema de $|V|$ clases a uno binario.

### La capa Oculta y los Embeddings

Al terminar el entrenamiento, ¿Qué nos queda en la capa oculta?

Una matriz de $v$ filas por $d$ columnas, la cual contiene lo que buscabamos: Una representación continua de todas las palabras de nuestro vocabulario.  

**Cada fila de la matriz es un vector que contiene la representación continua una palabra del vocabulario.**


<img src="http://mccormickml.com/assets/word2vec/word2vec_weight_matrix_lookup_table.png" alt="Capa Oculta 1" style="width: 400px;"/>

¿Cómo la usamos eficientemente?

Simple: usamos los mismos vectores one-hot de la entrada y las multiplicamos por la matriz:

<img src="http://mccormickml.com/assets/word2vec/matrix_mult_w_one_hot.png" alt="Skip Gram" style="width: 400px;"/>

### Visualización

Veamos cómo se ven los embeddings de Word2Vec entrenados sobre un corpus gigante en Inglés. Para facilitar el análisis se reducen las 200 dimensiones a 3. El link a la visualización es el siguiente: Visualización: https://projector.tensorflow.org/

### Espacio multidimensional

Teniendo nuestro embeddings entonces podríamos hacer operaciones tan interesantes como las siguientes:

Manzana + Púrpura -> Ciruela

Rey - Hombre + Mujer -> Reina

Si bien no es posible obtener exactamente dichos vectores, esperaríamos que las palabras más cercanas al vector resultante serían las entregadas, obteniendo así un significado de las palabras según su contexto.


### Fuentes

Word2vec:
- mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/
- https://towardsdatascience.com/introduction-to-word-embedding-and-word2vec-652d0c2060fa

Gensim:
- https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial

Nota: Las últimas 2 imagenes pertenecen a [Chris McCormick](http://mccormickml.com/about/)


In [1]:
# Contextualized word embeddings: BERT, ELMO, FLAIR.

x = 'El banco estaba lleno.'
y = 'El banco de sangre necesita personal.'

## **Entrenar nuestros Embeddings**

Para entrenar nuestros embeddings, usaremos el paquete gensim. Este trae una muy buena implementación de `word2vec`.




In [2]:
import re
import pandas as pd
from time import time
from collections import defaultdict
import string
import multiprocessing
import os
import requests
import numpy as np

# word2vec
from gensim.models import Word2Vec, KeyedVectors
from gensim.models.phrases import Phrases, Phraser

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

# scikit-learn
from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator, TransformerMixin

# visualizaciones
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import widgets

### Cargar el dataset y limpiar

Utilizaremos un datos de diversas fuentes de noticias que cuenta con tres atributos principales:

* texto: Referente al texto de la noticia.
* medio: Referente al medio de la noticia.
* fecha: Referente a la fecha de publicación.

In [3]:
dataset = pd.read_csv('https://raw.githubusercontent.com/giturra/JCC2023-WE/main/noticias_oct_dic_2019.tsv', sep='\t')

In [4]:
dataset.head()

,texto,medio,fecha
0,Rey de Tailandia despoja de títulos a su conso...,El Mercurio,2019-10-21 14:51:00
1,Denuncian que personas con epilepsia sufrieron...,El Mercurio,2019-12-18 12:12:00
2,Índice victimización de hogares en el país suf...,El Mercurio,2019-10-15 09:58:00
3,"Meditación, cocina, jardinería, manualidades: ...",El Mercurio,2019-11-16 16:42:00
4,Director de empresa que procesa datos electora...,El Mercurio,2019-11-01 05:25:00


In [5]:
# unir titulo con contenido de la noticia
content = dataset['texto']

In [6]:
content.head()

,texto
0,Rey de Tailandia despoja de títulos a su conso...
1,Denuncian que personas con epilepsia sufrieron...
2,Índice victimización de hogares en el país suf...
3,"Meditación, cocina, jardinería, manualidades: ..."
4,Director de empresa que procesa datos electora...


In [7]:
import tensorflow as tf
tf.__version__

'2.17.1'

In [8]:
content.shape

(10000,)

In [9]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [10]:
from collections import Counter

# limpiar puntuaciones y separar por tokens.
punctuation = string.punctuation + "«»“”‘’…—"
stopwords = pd.read_csv(
    'https://raw.githubusercontent.com/Alir3z4/stop-words/master/spanish.txt'
).values
stopwords = Counter(stopwords.flatten().tolist())

def simple_tokenizer(doc, lower=False):
    if lower:
        tokenized_doc = doc.translate(str.maketrans(
            '', '', punctuation)).lower().split()

    tokenized_doc = doc.translate(str.maketrans('', '', punctuation)).split()
    tokenized_doc = [
        token for token in tokenized_doc if token.lower() not in stopwords
    ]
    return tokenized_doc

cleaned_content = [simple_tokenizer(doc) for doc in content.values]

In [11]:
print("Ejemplo de alguna noticia: {}".format(cleaned_content[14]))

Ejemplo de alguna noticia: ['Ley', 'Ciudadanía', 'polémica', 'norma', 'india', 'acusan', 'discriminatoria', 'provocado', 'oleada', 'protestas', 'semana', 'miles', 'personas', 'salido', 'a', 'manifestarse', 'semana', 'consecutiva', 'ciudades', 'India', 'mostrar', 'rechazo', 'Ley', 'Ciudadanía', 'CAB', 'migrantes', 'auspiciada', 'Gobierno', 'jornadas', 'terminado', 'enfrentamiento', 'marchas', 'vuelto', 'violentas', 'registran', '15', 'muertes', 'relacionadas', 'hechos', 'Pero¿por', 'rechazo', 'amplio', 'a', 'ley', 'gubernamental', 'ley', 'Ciudadanía', 'viene', 'a', 'reemplazar', 'a', 'norma', 'vigente', '64', 'años', 'prohíbe', 'migrantes', 'ilegales', 'pasen', 'a', 'ciudadanos', 'India', 'BBC', 'antigua', 'legislación', 'ilegales', 'a', 'inmigrantes', 'ingresan', 'país', 'pasaporte', 'vigente', 'documento', 'viaje', 'a', 'quedan', 'India', 'permitido', 'personas', 'deportadas', 'detenidas', 'ley', 'aprobó', '11', 'diciembre', 'anula', 'normativa', 'señala', 'persona', 'vivir', 'Gobiern

### Extracción de Frases

Para crear buenas representaciones, es necesario tambien encontrar conjuntos de palabras que por si solas no tengan mayor significado (como `nueva` y `york`), pero que juntas que representen ideas concretas (`nueva york`).

Para esto, usaremos el primer conjunto de herramientas de `gensim`: `Phrases` y `Phraser`.

In [12]:
# Phrases recibe una lista de oraciones, y junta bigramas que estén al menos 100 veces repetidos
# como un único token. Detrás de esto hay un modelo estadístico basado en frecuencias, probabilidades, etc
# pero en términos simples ese es el resultado

phrases = Phrases(cleaned_content, min_count=100, progress_per=5000)

Ahora, usamos `Phraser` para re-tokenizamos el corpus con los bigramas encontrados. Es decir, juntamos los tokens separados que detectamos como frases.

In [13]:
bigram = Phraser(phrases)
sentences = bigram[cleaned_content]

In [14]:
# para ver como quedan las noticias retokenizadas, quitar comentario a la siguiente linea:
print(sentences[1])

['Denuncian', 'personas', 'epilepsia', 'sufrieron', 'ataques', 'Twitter', 'Usuarios', 'publicaron', 'luces', 'estroboscópicas', 'organización', 'Epilepsy', 'Foundation', 'origen', 'estadounidense', 'presentó', 'denuncia', 'penal', 'solicitó', 'investigación', 'a', 'raíz', 'distintos', 'usuarios', 'Twitter', 'publicaran', 'videos', 'luces', 'intermitentes', 'estroboscópicas', 'noviembre', 'provocaron', 'convulsiones', 'a', 'personas', 'epilepsia', 'usuarios', 'plataforma', 'utilizaron', 'hashtag', 'fundación', 'atacar', 'a', 'personas', 'epilepsia', 'acuerdo', 'a', 'indicado', 'directora', 'defensa', 'legal', 'Allison', 'Nichol', 'a', 'BBC', 'ataques', 'a', 'persona', 'luz', 'estroboscópica', 'a', 'convención', 'personas', 'epilepsia', 'convulsiones', 'intención', 'inducir', 'convulsiones', 'causar', 'daño', 'significativo', 'a', 'participantes', 'ataques', 'ocurrieron', 'Mes', 'Nacional', 'Concientización', 'Epilepsia', 'resalta', 'naturaleza', 'reprensible', 'encargada', 'materia', 'l

### Definir el modelo



Primero, como es usual, creamos el modelo. En este caso, usaremos uno de los primero modelos de embeddings neuronales: `word2vec`

Algunos parámetros importantes:

- `min_count`: Ignora todas las palabras que tengan frecuencia menor a la indicada.
- `window` : Tamaño de la ventana. Usaremos 4.
- `size` : El tamaño de los embeddings que crearemos. Por lo general, el rendimiento sube cuando se usan mas dimensiones, pero después de 300 ya no se nota cambio. Ahora, usaremos solo 200.
- `workers`: Cantidad de CPU que serán utilizadas en el entrenamiento.

In [15]:
w2v = Word2Vec(min_count=10,
                      window=4,
                      vector_size=200,
                      sample=6e-5,
                      alpha=0.03,
                      min_alpha=0.0007,
                      negative=20,
                      workers=multiprocessing.cpu_count())

In [16]:
Word2Vec()

### Construir el vocabulario

Para esto, se creará un conjunto que contendrá (una sola vez) todas aquellas palabras que aparecen mas de `min_count` veces.

In [17]:
w2v.build_vocab(sentences, progress_per=10000)

### Entrenar el Modelo

A continuación, entenaremos el modelo.
Los parámetros que usaremos serán:

- `total_examples`: Número de documentos.
- `epochs`: Número de veces que se iterará sobre el corpus.

Es recomendable que tengan instalado `cpython` antes de continuar. Aumenta bastante la velocidad de entrenamiento.


In [18]:
t = time()
w2v.train(sentences, total_examples=w2v.corpus_count, epochs=5, report_delay=10)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to train the model: 0.59 mins


Ahora que terminamos de entrenar el modelo, le indicamos que no lo entrenaremos mas.
Esto nos permitirá ejecutar eficientemente las tareas que realizaremos.

In [19]:
w2v.init_sims(replace=True)

<ipython-input-19-dc91b3835df3>:1: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v.init_sims(replace=True)


###  Guardar y cargar el modelo

Para ahorrar tiempo, usaremos un modelo preentrenado.

In [20]:
# Si entrenaste el modelo y lo quieres guardar, descomentar el siguiente bloque.
if not os.path.exists('./pretrained_models'):
    os.mkdir('./pretrained_models')
w2v.save('./pretrained_models/biobio_w2v.model')


# cargar el modelo (si es que lo entrenaron desde local.)
w2v = KeyedVectors.load("./pretrained_models/biobio_w2v.model", mmap='r')


In [21]:
# descargar el modelo desde github
def read_model_from_github(url):
    if not os.path.exists('./pretrained_models'):
        os.mkdir('./pretrained_models')

    r = requests.get(url)
    filename = url.split('/')[-1]
    with open('./pretrained_models/' + filename, 'wb') as f:
        f.write(r.content)
    return True


[
    read_model_from_github(file) for file in [
        'https://github.com/dccuchile/CC6205/releases/download/Data/biobio_w2v.model',
    ]
]
# cargar el modelo (si es que lo entrenaron desde local.)
w2v = KeyedVectors.load("./pretrained_models/biobio_w2v.model", mmap='r')


## **Tasks: Palabras mas similares y Analogías**

### **Palabras mas similares**

Tal como dijimos anteriormente, los embeddings son capaces de codificar toda la información contextual de las palabras en vectores.

Y como cualquier objeto matemático, estos pueden operados para encontrar ciertas propiedades. Tal es el caso de las  encontrar las palabras mas similares, lo que no es mas que encontrar los n vecinos mas cercanos del vector.  

In [22]:
w2v.wv.most_similar(positive=["perro"])

[('gato', 0.7324815392494202),
 ('perrito', 0.7017662525177002),
 ('cachorro', 0.6726856231689453),
 ('canino', 0.6614428758621216),
 ('mascota', 0.6354753971099854),
 ('animal', 0.6341222524642944),
 ('gatito', 0.6259311437606812),
 ('felino', 0.622412919998169),
 ('perros', 0.6207762360572815),
 ('perra', 0.5834704041481018)]

In [23]:
w2v.wv.most_similar(positive=["Chile"])

[('Latinoamérica', 0.48114773631095886),
 ('país', 0.4791499078273773),
 ('exportadores', 0.4531676173210144),
 ('chileno', 0.4481710195541382),
 ('Crecimiento', 0.4425714910030365),
 ('posiciona', 0.44039905071258545),
 ('Perú', 0.43912073969841003),
 ('Bolivia', 0.4357107877731323),
 ('chilenas', 0.43546050786972046),
 ('chilena', 0.430389940738678)]

In [24]:
w2v.wv.most_similar(positive=["Bolsonaro"])

[('ultraderechista_Jair', 0.7692869305610657),
 ('excapitán_Ejército', 0.7673073410987854),
 ('Jair_Bolsonaro', 0.7401224374771118),
 ('Haddad', 0.6947491765022278),
 ('exmilitar', 0.6393497586250305),
 ('Brasil', 0.618027925491333),
 ('Brasilia', 0.6159982681274414),
 ('nostálgico', 0.6144426465034485),
 ('Fernando_Haddad', 0.606082558631897),
 ('Ultraderechista', 0.603746771812439)]

In [25]:
w2v.wv.most_similar(positive=["Trump"])

[('Casa_Blanca', 0.709988534450531),
 ('Donald_Trump', 0.7093803882598877),
 ('mandatario_estadounidense', 0.7046915292739868),
 ('inquilino', 0.678024411201477),
 ('administración_Trump', 0.6302800178527832),
 ('Bolton', 0.6201831102371216),
 ('Unidos', 0.6197685599327087),
 ('Washington', 0.6131006479263306),
 ('presidente_estadounidense', 0.6014897227287292),
 ('Unidos_Donald', 0.5976207852363586)]

In [26]:
w2v.wv.most_similar(positive=["Boric"])

[('Silber', 0.7718173265457153),
 ('Ascencio', 0.7053338289260864),
 ('Orsini', 0.6885764002799988),
 ('democratacristiano', 0.6543008089065552),
 ('Giorgio', 0.6531973481178284),
 ('Jiles', 0.6507138013839722),
 ('frenteamplista', 0.6495876312255859),
 ('Crispi', 0.6427950859069824),
 ('Gabriel', 0.6346496343612671),
 ('Cariola', 0.6257592439651489)]

In [27]:
w2v.wv.most_similar(positive=["Uber"])

[('Cabify', 0.7779495716094971),
 ('Eats', 0.700801432132721),
 ('DiDi', 0.676181435585022),
 ('Rappi', 0.6054359078407288),
 ('Conductores', 0.5858488082885742),
 ('aplicaciones', 0.5700352787971497),
 ('Beat', 0.566635251045227),
 ('app', 0.5640519857406616),
 ('choferes', 0.5415353178977966),
 ('taxistas', 0.5351073145866394)]

In [28]:
w2v.wv.most_similar(positive=["Huawei"])

[('ZTE', 0.7025798559188843),
 ('telecomunicaciones', 0.639496922492981),
 ('Wanzhou', 0.617560863494873),
 ('5G', 0.6000877022743225),
 ('Meng', 0.5700865387916565),
 ('Pekin', 0.5595781803131104),
 ('Ren', 0.5414323210716248),
 ('gigante_asiático', 0.5363011956214905),
 ('sanciones_estadounidenses', 0.5254279375076294),
 ('China', 0.5246084928512573)]

In [29]:
w2v.wv.most_similar(positive=["TVN"])

[('Televisión', 0.6506631374359131),
 ('directorio', 0.5564517974853516),
 ('Orrego', 0.48779889941215515),
 ('C5N', 0.47816959023475647),
 ('Panorama', 0.47550544142723083),
 ('Chilevisión', 0.46353694796562195),
 ('cuprífera', 0.4621235728263855),
 ('matinal', 0.45264914631843567),
 ('CHV', 0.4483409821987152),
 ('Mega', 0.44625139236450195)]

### **Analogías**

Por otra parte, la analogía consiste en comparar 3 terminos mediante una operación del estilo:

$$palabra1 - palabra2 \approx palabra 3 - x$$

para encontrar relaciones entre estos.

Por ejemplo:

| palabra 1 (pos) |  palabra 2 (neg) |
|-----------------|------------------|
|  macri          | Argentina          |
| Brasil           |  x               |

In [30]:
w2v.wv.most_similar(positive=["Macri", "Brasil"], negative=['Argentina'], topn=10)

[('Bolsonaro', 0.6208245158195496),
 ('ultraderechista_Jair', 0.5999900698661804),
 ('Jair_Bolsonaro', 0.5594528913497925),
 ('Michel_Temer', 0.5592767596244812),
 ('excapitán_Ejército', 0.556822657585144),
 ('Haddad', 0.532974123954773),
 ('Fernando_Haddad', 0.5272845029830933),
 ('Temer', 0.5139399766921997),
 ('Brasilia', 0.4959222972393036),
 ('Ultraderechista', 0.47941094636917114)]

In [31]:
w2v.wv.most_similar(positive=["Chile", "Huawei"], negative=['China'], topn=10)

[('Falabella', 0.47030484676361084),
 ('portabilidad', 0.4300999641418457),
 ('Multicaja', 0.4203709363937378),
 ('CMR', 0.41495582461357117),
 ('Sodimac', 0.4064332842826843),
 ('Valledor', 0.3986559212207794),
 ('Telefónica', 0.39562681317329407),
 ('Ciberseguridad', 0.39511701464653015),
 ('Subtel', 0.3942727744579315),
 ('retail', 0.39312079548835754)]

In [32]:
# atención
w2v.wv.most_similar(positive=["perro", "tigre"], negative=['gato'], topn=10)

[('elefante', 0.5690402388572693),
 ('macho', 0.5560784935951233),
 ('cría', 0.5545395016670227),
 ('hembra', 0.5471694469451904),
 ('domador', 0.5464577674865723),
 ('amarrado', 0.5397398471832275),
 ('cuernos', 0.5395569801330566),
 ('canguro', 0.5395484566688538),
 ('tigres', 0.5384529829025269),
 ('hueso', 0.5378432273864746)]

## **Word Embeddings como características para clasificar**


En esta sección, veremos como utilizar los word embeddings como característica para **clasificar noticias de diferentes medios**.


### Dividir el dataset en training y test

In [33]:
X_train, X_test, y_train, y_test = train_test_split(dataset.texto,
                                                    dataset.medio,
                                                    test_size=0.33,
                                                    random_state=42)

Primero, crearemos el Transformer con el cual convertiremos el documento a vector.


### Doc2vec

In [34]:
class Doc2VecTransformer(BaseEstimator, TransformerMixin):
    """ Transforma tweets a representaciones vectoriales usando algún modelo de Word Embeddings.
    """

    def __init__(self, model, aggregation_func):
        # extraemos los embeddings desde el objeto contenedor. ojo con esta parte.
        self.model = model.wv

        # indicamos la función de agregación (np.min, np.max, np.mean, np.sum, ...)
        self.aggregation_func = aggregation_func

    def simple_tokenizer(self, doc, lower=False):
        """Tokenizador. Elimina signos de puntuación, lleva las letras a minúscula(opcional) y
           separa el tweet por espacios.
        """
        if lower:
            doc.translate(str.maketrans('', '', string.punctuation)).lower().split()
        return doc.translate(str.maketrans('', '', string.punctuation)).split()

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):

        doc_embeddings = []

        for doc in X:
            # tokenizamos el documento. Se llevan todos los tokens a minúscula.
            # ojo con esto, ya que puede que tokens con minúscula y mayúscula tengan
            # distintas representaciones
            tokens = self.simple_tokenizer(doc, lower = True)

            selected_wv = []
            for token in tokens:
                if token in self.model.index_to_key:
                    selected_wv.append(self.model[token])

            # si seleccionamos por lo menos un embedding para el tweet, lo agregamos y luego lo añadimos.
            if len(selected_wv) > 0:
                doc_embedding = self.aggregation_func(np.array(selected_wv), axis=0)
                doc_embeddings.append(doc_embedding)
            # si no, añadimos un vector de ceros que represente a ese documento.
            else:
                print('No pude encontrar ningún embedding en el tweet: {}. Agregando vector de ceros.'.format(doc))
                doc_embeddings.append(np.zeros(self.model.vector_size)) # la dimension del modelo

        return np.array(doc_embeddings)


### Definimos el pipeline


Usaremos la transformación que creamos antes mas una regresión logística.

In [35]:
clf = LogisticRegression(max_iter=1000000)

doc2vec_mean = Doc2VecTransformer(w2v, np.mean)
doc2vec_sum = Doc2VecTransformer(w2v, np.sum)
doc2vec_max = Doc2VecTransformer(w2v, np.max)


pipeline = Pipeline([('doc2vec', doc2vec_sum), ('clf', clf)])

In [36]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('doc2vec',
                 Doc2VecTransformer(aggregation_func=<function sum at 0x797141bf2330>,
                                    model=<gensim.models.keyedvectors.KeyedVectors object at 0x7970a19d3a00>)),
                ('clf', LogisticRegression(max_iter=1000000))])

**Predecimos y evaluamos:**

In [37]:
y_pred = pipeline.predict(X_test)

In [38]:
conf_matrix = confusion_matrix(y_test, y_pred)
print(conf_matrix)

[[154  19  16  13   7  19  26  18  28  15]
 [ 13 206  25   9   5  22  24  12   9  10]
 [ 22  39 107  15  23  55  37  20  10  17]
 [  5   5  12 224  33  18   5   4  12   5]
 [ 10  11  22  44 219  23   7   5   6   4]
 [ 21  35  50  17  29  96  15  14  20  22]
 [  3   6   5   1   0   1 245   9  28   9]
 [ 20  19  12   4   7  13  10 219  21   7]
 [ 31   2  14   4   3  14  35  16 191  10]
 [ 27  11  13  10  11  18  31  11  13 208]]


In [39]:
print(classification_report(y_test, y_pred))

                 precision    recall  f1-score   support

            CHV       0.50      0.49      0.50       315
 Cooperativa CL       0.58      0.61      0.60       335
    El Mercurio       0.39      0.31      0.34       345
  El Rancaguino       0.66      0.69      0.67       323
   La Discusión       0.65      0.62      0.64       351
      La Nación       0.34      0.30      0.32       319
         La RED       0.56      0.80      0.66       307
           MEGA       0.67      0.66      0.66       332
Radio Concierto       0.57      0.60      0.58       320
     The Clinic       0.68      0.59      0.63       353

       accuracy                           0.57      3300
      macro avg       0.56      0.57      0.56      3300
   weighted avg       0.56      0.57      0.56      3300



In [40]:
pipeline.predict(
    [("Alguna noticia..")])

array(['La RED'], dtype=object)

## **Usandolo con BoW**

In [41]:
X_train, X_test, y_train, y_test = train_test_split(dataset.texto,
                                                    dataset.medio,
                                                    test_size=0.33,
                                                    random_state=42)

In [42]:
# Definimos el vectorizador para convertir el texto a BoW:
vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 2))

# Definimos el clasificador que usaremos.
clf_2 = LogisticRegression(max_iter=10000)

# Definimos el pipeline
pipeline_2 = Pipeline([('features',
                        FeatureUnion([('bow', CountVectorizer()),
                                      ('doc2vec', doc2vec_sum)])), ('clf', clf)])

In [43]:
pipeline_2.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('bow', CountVectorizer()),
                                                ('doc2vec',
                                                 Doc2VecTransformer(aggregation_func=<function sum at 0x797141bf2330>,
                                                                    model=<gensim.models.keyedvectors.KeyedVectors object at 0x7970a19d3a00>))])),
                ('clf', LogisticRegression(max_iter=1000000))])

In [44]:
y_pred_2 = pipeline_2.predict(X_test)
conf_matrix = confusion_matrix(y_test, y_pred_2)
print(conf_matrix)

[[252   4  11   2   2   7   4   2  28   3]
 [  3 298  13   2   3  10   1   3   1   1]
 [ 10  11 196   9  18  50  19  12  10  10]
 [  1   3   6 274  10  17   3   3   4   2]
 [  1   0  20  13 277  26   4   4   0   6]
 [  3   7  55  12  16 172  13   6  13  22]
 [  1   1   9   0   0   0 271   3  14   8]
 [  1  10  12   1   5   7   8 274   7   7]
 [ 27   0   6   1   1   8  13   5 251   8]
 [  6   5  14  10   6  25  13   2   7 265]]


In [45]:
print(classification_report(y_test, y_pred_2))

                 precision    recall  f1-score   support

            CHV       0.83      0.80      0.81       315
 Cooperativa CL       0.88      0.89      0.88       335
    El Mercurio       0.57      0.57      0.57       345
  El Rancaguino       0.85      0.85      0.85       323
   La Discusión       0.82      0.79      0.80       351
      La Nación       0.53      0.54      0.54       319
         La RED       0.78      0.88      0.83       307
           MEGA       0.87      0.83      0.85       332
Radio Concierto       0.75      0.78      0.77       320
     The Clinic       0.80      0.75      0.77       353

       accuracy                           0.77      3300
      macro avg       0.77      0.77      0.77      3300
   weighted avg       0.77      0.77      0.77      3300



### Propuesto...

- Usar su modelo de embeddings favorito para ver si mejora la clasificación:
    
 - Fast y word2vec en español, [cortesía](https://github.com/dccuchile/spanish-word-embeddings) de los grandes del DCC
 - [Conceptnet](https://github.com/commonsense/conceptnet-numberbatch)


- Visualizar los documentos usando `doc2vec`